### # Topic modeling  ## US Presidents' State of the Union Addresses and Messages Data: http://www.presidency.ucsb.edu/sou.php Inspiration: https://www.exaptive.com/blog/topic-modeling-the-state-of-the-union

In [1]:
from bs4 import BeautifulSoup
import urllib2
import json
import re
url = "http://www.presidency.ucsb.edu/sou.php"
handle=urllib2.urlopen(url);
html_text=handle.read();
soup=BeautifulSoup(html_text, "html.parser");
soup
anchors=soup.find_all("td", class_="ver12");

In [2]:
#Get a dictionary {year: url}
messages = {}
for anch in soup.findAll('a'):
    if anch.has_attr('href') and '.edu/ws/index.php' in anch["href"] and anch.text.isdigit(): 
        if int(anch.text) in range(1935,2018):
            messages[int(anch.text)]=anch["href"]

print ""
print "NB: Nixon made some special messages. Roosevelt made two versions"


NB: Nixon made some special messages. Roosevelt made two versions


In [3]:
# Scrape all messages to soups dictionary
if 'soups' not in locals(): soups = {}
for year,url in messages.iteritems(): #"http://www.presidency.ucsb.edu/ws/index.php?pid=51634"
    if year in range(1945,2018) and year not in soups.keys():
        handle=urllib2.urlopen(url);
        html_text=handle.read()
        html_text = re.sub(u'<br>|</?[up]>', " ", html_text)
        html_text = re.sub(u'<[a-z]{1,4}>[^a-zA-Z]*</[a-z]{1,4}>', "", html_text)
        soups[year] = BeautifulSoup(html_text, "html.parser");

In [4]:
# Create documents_raw list, presidents[year] and texts[year] dictionaries
import pandas as pd
documents_raw = list()
presidents = {}
texts = {}
for year,soup in soups.iteritems():
    span = soup.find_all("span", class_="displaytext");
    text = span[0].text
    titles = soup.find_all("title")
    title = titles[0].text
    pres = title.split(":")[0]
    presidents[year] = pres
    texts[year] = text
    documents_raw.append(text)
    
df_corpus = pd.DataFrame({'Presidents': presidents, 'Text': texts}) #'Year': soups.keys(), 
df_corpus.head()

,Presidents,Text
1945,Franklin D. Roosevelt,To the Congress: In considering the State of ...
1946,Harry S. Truman,"[Released January 21, 1946. Dated January 14, ..."
1947,Harry S. Truman,"Mr. President, Mr. Speaker, Members of the Con..."
1948,Harry S. Truman,"Mr. President, Mr. Speaker, and Members of the..."
1949,Harry S. Truman,"Mr. President, Mr. Speaker, Members of the Con..."


In [5]:
#Dump documents_raw to file
import pickle
output = open('documents_raw.pkl', 'wb')
pickle.dump(documents_raw, output)
output.close()
#Import as: 
#pkl_file = open('documents_raw.pkl', 'rb')
#documents_raw = pickle.load(pkl_file)

### Double check documents_raw 
* No < or > symbols anymore
* All numbers followed by letters are for now ok to keep

In [35]:
for i,doc in enumerate(documents_raw):
    check = re.findall('<|>',doc)
    if len(check): print i,check
    check = re.findall('(.{5}[0-9][a-zA-Z].{5})',doc)
    if len(check): print i,check

1 [u'98 (79th Con']
2 [u'rch 31st. Th']
3 [u'the 80th Con']
4 [u'his 81st Con', u'the 80th Con', u'the 81st Con']
5 [u'the 20th cen', u'the 20th cen', u'the 20th cen']
6 [u'his 82d Cong', u'the 82d Cong']
9 [u'pril 1st, an', u'ary 14th.  H']
10 [u'the 84th Con', u'une 30th. Fo', u'the 83rd Con', u'the 84th Con', u'the 20th of ', u'ary 27th det', u'the 83rd Con', u'the 83rd Con', u'the 83rd Con']
11 [u'pril 1st.  I']
12 [u'the 85th Con', u'the 85th Con']
13 [u'the 85th Con', u'the 85th Con']
14 [u'the 86th Con', u'he 170th yea']
15 [u'the 86th Con']
16 [u'our 49th and']
17 [u'the 87th Con', u'he 100th ann']
18 [u'the 88th Con', u'he 175th yea', u' the 5th gra']
20 [u'the 20th cen']
21 [u'the 89th Con']
22 [u'The 88th and', u'the 89th Con', u'the 90th Con', u'the 90th Con', u'the 90th Con', u'The 89th Con', u'the 90th Con']
23 [u'nks 15th amo', u' our 8th yea', u'uary 1st, I ', u"'s 200th bir", u'ur 200th bir']
24 [u'nks 15th amo', u'the 13th, 14', u'and 15th ame']
25 [u'ts 200th bir', u